In [1]:
import tensorflow as tf
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tf_keras_vis.gradcam import Gradcam

In [ ]:
# Load and preprocess images
data_dir = "brain_mri_dataset"
images, labels = [], []
for label in ["tumor", "no_tumor"]:
    folder = os.path.join(data_dir, label)
    for img_name in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, img_name), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 128)) / 255.0
        images.append(img)
        labels.append(1 if label == "tumor" else 0)

images = np.array(images).reshape(-1, 128, 128, 1)
labels = np.array(labels)

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Data augmentation
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import MobileNetV2

In [ ]:
def build_model():
    base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
    base_model.trainable = False
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(128, 128, 1)),
        tf.keras.layers.Lambda(lambda x: tf.image.grayscale_to_rgb(x)),
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    return model

model = build_model()
plot_model(model, to_file="mobilenetv2_architecture.png", show_shapes=True, show_layer_names=True)